In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking
import ast

In [58]:
df = pd.read_csv('prepared_data/back.csv')
df

,Unnamed: 0,Category,Accelerometer,Gyroscope,Magnetometer,Duration
0,0,CLIMBING,"[[-6.850950241088867, 6.60105037689209, -1.287...","[[0.29122498631477356, 0.5505499839782715, 0.2...","[[65.8125, -19.012500762939453, 16.80000114440...",20
1,1,FALLING,"[[-5.451000213623047, 7.82895040512085, 1.9920...","[[-0.07741250097751617, 0.2190375030040741, -0...","[[32.34375, -38.94375228881836, -29.3437519073...",70
2,2,SUCCESSFULL,"[[6.903000354766846, -1.7880001068115234, -4.9...","[[-0.23718750476837158, 0.36121249198913574, 0...","[[-37.91250228881836, 10.143750190734863, -6.3...",85
3,3,WALKING,"[[-6.942000389099121, 5.0400004386901855, -0.0...","[[0.1816374957561493, -0.605412483215332, 0.25...","[[34.14375305175781, -34.20000076293945, 15.95...",25
4,4,CLIMBING,"[[-6.130050182342529, 6.98805046081543, 2.9569...","[[0.11261250078678131, 0.16541250050067902, -0...","[[30.187501907348633, -6.056250095367432, -4.1...",50
5,5,FALLING,"[[-5.57295036315918, 7.07205057144165, 1.01295...","[[0.46131250262260437, 0.5150749683380127, -0....","[[32.34375, -40.38750076293945, -23.2875003814...",2
6,6,SUCCESSFULL,"[[6.709050178527832, -7.123950481414795, -0.58...","[[0.08277499675750732, -0.210099995136261, -0....","[[-26.45625114440918, 22.087501525878906, -27....",85
7,7,WALKING,"[[-3.0010502338409424, 7.975050449371338, 0.62...","[[-0.16554999351501465, -1.0306999683380127, -...","[[21.11250114440918, -35.75625228881836, 12.90...",25
8,8,CLIMBING,"[[-7.414050579071045, 7.0819501876831055, 1.23...","[[0.489362508058548, -0.05211250111460686, -0....","[[32.04375076293945, -19.743751525878906, -18....",60
9,9,FALLING,"[[-4.780050277709961, 6.909000396728516, 0.280...","[[-0.20446249842643738, 0.1387374997138977, 0....","[[37.80000305175781, -22.856250762939453, -12....",2


In [59]:
# Convert string to list of list of floats
df['Accelerometer'] = df['Accelerometer'].apply(eval)
df['Gyroscope'] = df['Gyroscope'].apply(eval)
df['Magnetometer'] = df['Magnetometer'].apply(eval)

# Find the maximum length among the sequences
max_length = max(
    df['Accelerometer'].apply(len).max(),
    df['Gyroscope'].apply(len).max(),
    df['Magnetometer'].apply(len).max()
)

In [60]:

# Create empty arrays to store the masked sequences
X_accelerometer = np.zeros((len(df), max_length, 3))
X_gyroscope = np.zeros((len(df), max_length, 3))
X_magnetometer = np.zeros((len(df), max_length, 3))

# Mask the sequences with zeros
for i, (acc, gyr, mag) in enumerate(zip(df['Accelerometer'], df['Gyroscope'], df['Magnetometer'])):
    X_accelerometer[i, :len(acc)] = acc
    X_gyroscope[i, :len(gyr)] = gyr
    X_magnetometer[i, :len(mag)] = mag


# Split data into features (X) and labels (y)
y = df['Category'].values

# Encode labels as numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_accelerometer, y, test_size=0.2, random_state=42)

# Create the LSTM model
model = Sequential()
model.add(Masking(mask_value=0.0, input_shape=(max_length, 3)))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1/1 [==============================] - 5s 5s/step - loss: 0.3433 - accuracy: 0.1600
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 0.1454 - accuracy: 0.2000
Epoch 3/10
1/1 [==============================] - 1s 1s/step - loss: -0.0492 - accuracy: 0.2400
Epoch 4/10
1/1 [==============================] - 1s 1s/step - loss: -0.2404 - accuracy: 0.2400
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: -0.4271 - accuracy: 0.2400
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: -0.6073 - accuracy: 0.2400
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: -0.7789 - accuracy: 0.2400
Epoch 8/10
1/1 [==============================] - 1s 1s/step - loss: -0.9416 - accuracy: 0.2400
Epoch 9/10
1/1 [==============================] - 1s 1s/step - loss: -1.0971 - accuracy: 0.2400
Epoch 10/10
1/1 [==============================] - 1s 1s/step - loss: -1.2478 - accuracy: 0.2400


In [63]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)
model.evaluate

1/1 [==============================] - 0s 277ms/step - loss: -0.2127 - accuracy: 0.4286
Test Loss: -0.21271459758281708
Test Accuracy: 0.4285714328289032


<bound method Model.evaluate of <keras.engine.sequential.Sequential object at 0x147d09950>>